In [ ]:
%pip install pycoingecko
%pip install plotly
%pip install mplfinance
%pip install --upgrade nbformat

In [46]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [50]:
cg = CoinGeckoAPI()
bitcoin_data = cg.get_coin_market_chart_by_id(id="bitcoin", vs_currency="usd", days=30)

bitcoin_data["prices"][:10]

[[1756652501914, 108462.07683097602],
 [1756656141408, 108786.16404496257],
 [1756659637043, 108949.12666027468],
 [1756663271785, 108979.48633378587],
 [1756666912435, 109131.67937094519],
 [1756670432327, 108922.73073924262],
 [1756674157565, 109066.36655120115],
 [1756677698716, 109026.24716432432],
 [1756681330273, 108867.78018188462],
 [1756684884782, 108253.36092385623]]

In [51]:
bitcoin_price_data = bitcoin_data["prices"]
data = pd.DataFrame(bitcoin_price_data, columns=["TimeStamp", "Price"])

data

,TimeStamp,Price
0,1756652501914,108462.076831
1,1756656141408,108786.164045
2,1756659637043,108949.126660
3,1756663271785,108979.486334
4,1756666912435,109131.679371
...,...,...
716,1759230136030,113034.887542
717,1759233682003,113090.666326
718,1759237329316,113153.918953
719,1759240935094,113286.250464


In [52]:
# format date add date column
data["date"] = data["TimeStamp"].apply(
    lambda d: datetime.date.fromtimestamp(d / 1000.0)
)

data

,TimeStamp,Price,date
0,1756652501914,108462.076831,2025-08-31
1,1756656141408,108786.164045,2025-08-31
2,1756659637043,108949.126660,2025-08-31
3,1756663271785,108979.486334,2025-08-31
4,1756666912435,109131.679371,2025-08-31
...,...,...,...
716,1759230136030,113034.887542,2025-09-30
717,1759233682003,113090.666326,2025-09-30
718,1759237329316,113153.918953,2025-09-30
719,1759240935094,113286.250464,2025-09-30


In [53]:
candlestick_data = data.groupby(data.date, as_index=False).agg(
    {"Price": ["min", "max", "first", "last"]}
)
candlestick_data

date          Price                                             
                          min            max          first           last
0   2025-08-31  108462.076831  109131.679371  108462.076831  108922.730739
1   2025-09-01  107414.442250  109681.776742  109066.366551  109204.427254
2   2025-09-02  107541.658500  111219.975000  108788.136907  110752.303454
3   2025-09-03  110690.278112  112380.369638  111395.041655  112156.224118
4   2025-09-04  109497.723469  112230.160366  112230.160366  109873.893092
5   2025-09-05  110403.239035  113225.000213  110403.239035  111445.250207
6   2025-09-06  110054.773314  111619.386202  111619.386202  110054.773314
7   2025-09-07  110174.333714  111347.479269  110174.333714  111232.347963
8   2025-09-08  110836.313956  112601.637580  111325.774083  112067.347002
9   2025-09-09  110870.226491  113112.942019  112025.168573  111319.253012
10  2025-09-10  111023.099571  113934.396698  111359.195853  113402.845604
11  2025-09-11  113625.820288  114546.629069  113625.820288  114435.889092
12  2025-09-12  114402.348650  116704.714866  114402.348650  116704.714866
13  2025-09-13  115342.814107  116299.905741  116206.138077  115837.628677
14  2025-09-14  115264.810470  116143.869783  115975.606658  115673.898845
15  2025-09-15  114696.006061  116552.241074  115854.913508  115314.095872
16  2025-09-16  115019.095780  116804.438671  115315.757154  116804.438671
17  2025-09-17  115194.307804  117256.585491  116882.781701  115721.459931
18  2025-09-18  115649.823583  117851.185392  115649.823583  117479.654706
19  2025-09-19  115157.378419  117575.953057  117575.953057  115157.378419
20  2025-09-20  115436.270663  116083.240792  115436.270663  115743.814932
21  2025-09-21  115382.615839  115851.047616  115760.457645  115529.534806
22  2025-09-22  112176.155768  115584.691054  115369.305067  112176.155768
23  2025-09-23  111658.229488  113153.926988  112694.311547  111658.229488
24  2025-09-24  111715.174944  113874.186964  111866.496292  113488.982377
25  2025-09-25  109003.342952  113517.576693  113517.576693  109479.267830
26  2025-09-26  108963.530136  109855.697774  109170.430541  109258.922940
27  2025-09-27  109302.715768  109710.208747  109302.715768  109455.416770
28  2025-09-28  109340.758878  110358.271079  109417.497859  110358.271079
29  2025-09-29  110748.345984  114267.658155  110842.578688  114267.658155
30  2025-09-30  112890.221010  114591.640752  114335.555816  113486.095755

In [56]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=candlestick_data["date"],
            open=candlestick_data["Price"]["first"],
            high=candlestick_data["Price"]["max"],
            low=candlestick_data["Price"]["min"],
            close=candlestick_data["Price"]["last"],
        )
    ]
)

fig.update_layout(xaxis_rangeslider_visible=False)

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': {'bdata': ('z6EbsauX+kA52gfWRqn6QKuF8toECv' ... 'dWZPH6QISHzYe65ftABm02iOG0+0A='),
                        'dtype': 'f8'},
              'high': {'bdata': ('dxG03rqk+kBOZ4ltHMf6QKVCmZk/J/' ... 'dWZPH6QISHzYe65ftAwNaEQPr5+0A='),
                       'dtype': 'f8'},
              'low': {'bdata': ('FR6zOuF6+kBlxHQTZzn6QC8VN4laQf' ... 'wkzLH6QKhMJonFCftAEpxBiaOP+0A='),
                      'dtype': 'f8'},
              'open': {'bdata': ('FR6zOuF6+kDUymTdpaD6QNkYxTBCj/' ... 'v3l7b6QPnxTUKpD/tA8dme5Pjp+0A='),
                       'dtype': 'f8'},
              'type': 'candlestick',
              'x': array([datetime.date(2025, 8, 31), datetime.date(2025, 9, 1),
                          datetime.date(2025, 9, 2), datetime.date(2025, 9, 3),
                          datetime.date(2025, 9, 4), datetime.date(2025, 9, 5),
                          datetime.date(2025, 9, 6), datetime.date(2025, 9, 7),
                          datetime.date(2025, 9, 8), datetime.date(2025, 9, 9),
                          datetime.date(2025, 9, 10), datetime.date(2025, 9, 11),
                          datetime.date(2025, 9, 12), datetime.date(2025, 9, 13),
                          datetime.date(2025, 9, 14), datetime.date(2025, 9, 15),
                          datetime.date(2025, 9, 16), datetime.date(2025, 9, 17),
                          datetime.date(2025, 9, 18), datetime.date(2025, 9, 19),
                          datetime.date(2025, 9, 20), datetime.date(2025, 9, 21),
                          datetime.date(2025, 9, 22), datetime.date(2025, 9, 23),
                          datetime.date(2025, 9, 24), datetime.date(2025, 9, 25),
                          datetime.date(2025, 9, 26), datetime.date(2025, 9, 27),
                          datetime.date(2025, 9, 28), datetime.date(2025, 9, 29),
                          datetime.date(2025, 9, 30)], dtype=object)}],
    'layout': {'template': '...', 'xaxis': {'rangeslider': {'visible': False}}}
})